In [2]:
import polars as pl
import polars.selectors as cs
import time

start = time.time()
trips = pl.read_parquet("../data/taxi/yellow_tripdata_*.parquet")
sum_per_vendor = trips.group_by("VendorID").agg(cs.numeric().sum())
print(sum_per_vendor)
print(f"Time: {time.time() - start:.2f}s")

shape: (4, 16)
┌──────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ VendorID ┆ passenger ┆ trip_dist ┆ RatecodeI ┆ … ┆ improveme ┆ total_amo ┆ congestio ┆ airport_f │
│ ---      ┆ _count    ┆ ance      ┆ D         ┆   ┆ nt_surcha ┆ unt       ┆ n_surchar ┆ ee        │
│ i64      ┆ ---       ┆ ---       ┆ ---       ┆   ┆ rge       ┆ ---       ┆ ge        ┆ ---       │
│          ┆ f64       ┆ f64       ┆ f64       ┆   ┆ ---       ┆ f64       ┆ ---       ┆ f64       │
│          ┆           ┆           ┆           ┆   ┆ f64       ┆           ┆ f64       ┆           │
╞══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 6        ┆ 0.0       ┆ 526585.91 ┆ 0.0       ┆ … ┆ 17525.4   ┆ 2.7891e6  ┆ 0.0       ┆ 0.0       │
│ 1        ┆ 1.3536686 ┆ 3.6616e7  ┆ 2.4743708 ┆ … ┆ 3.5956e6  ┆ 2.3562e8  ┆ 2.5205e7  ┆ 861168.0  │
│          ┆ e7        ┆           ┆ e7        ┆   ┆           ┆           ┆

In [8]:
df = pl.read_parquet("../data/taxi/yellow_tripdata_2022-01.parquet")
print(df.columns)


['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag', 'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'congestion_surcharge', 'airport_fee']


In [19]:
income_per_distance_per_vendor = sum_per_vendor.select(
    "VendorID",
    income_per_distance=pl.col("total_amount") / pl.col("trip_distance"),
)

print(income_per_distance_per_vendor)

top_three = income_per_distance_per_vendor.sort(
    by="income_per_distance", descending=True
).head(3)
print(top_three)

shape: (4, 2)
┌──────────┬─────────────────────┐
│ VendorID ┆ income_per_distance │
│ ---      ┆ ---                 │
│ i64      ┆ f64                 │
╞══════════╪═════════════════════╡
│ 2        ┆ 3.117667            │
│ 1        ┆ 6.434789            │
│ 6        ┆ 5.296493            │
│ 5        ┆ 4.731557            │
└──────────┴─────────────────────┘
shape: (3, 2)
┌──────────┬─────────────────────┐
│ VendorID ┆ income_per_distance │
│ ---      ┆ ---                 │
│ i64      ┆ f64                 │
╞══════════╪═════════════════════╡
│ 1        ┆ 6.434789            │
│ 6        ┆ 5.296493            │
│ 5        ┆ 4.731557            │
└──────────┴─────────────────────┘


In [11]:
names_lf = pl.LazyFrame(
    {"name": ["Alice", "Bob", "Charlie"], "age": [25, 30, 36]}
)

erroneous_query = names_lf.columns(
    sliced_age=pl.col("age").str.slice(1, 3)
)

result_df = erroneous_query.collect()


C:\Users\valk\AppData\Local\Temp\ipykernel_12968\2987658003.py:5: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  erroneous_query = names_lf.columns(


TypeError: 'list' object is not callable

In [16]:
start = time.time()

trips = pl.scan_parquet("../data/taxi/yellow_tripdata_*.parquet")
sum_per_vendor = trips.group_by("VendorID").agg(cs.numeric().sum())

income_per_distance_per_vendor = sum_per_vendor.select(
    "VendorID",
    income_per_distance=pl.col("total_amount") / pl.col("trip_distance"),
)

top_three = income_per_distance_per_vendor.sort(
    by="income_per_distance", descending=True
).head(3)

print(top_three.collect())
print(f"Time: {time.time() - start:.2f}s")

naive plan: (run LazyFrame.explain(optimized=True) to see the optimized plan)

SLICE[offset: 0, len: 3]
  SORT BY [descending: [true]] [col("income_per_distance")]
    SELECT [col("VendorID"), [(col("total_amount")) / (col("trip_distance"))].alias("income_per_distance")]
      AGGREGATE[maintain_order: false]
        [col("passenger_count").sum(), col("trip_distance").sum(), col("RatecodeID").sum(), col("PULocationID").sum(), col("DOLocationID").sum(), col("payment_type").sum(), col("fare_amount").sum(), col("extra").sum(), col("mta_tax").sum(), col("tip_amount").sum(), col("tolls_amount").sum(), col("improvement_surcharge").sum(), col("total_amount").sum(), col("congestion_surcharge").sum(), col("airport_fee").sum()] BY [col("VendorID")]
        FROM
        Parquet SCAN [..\data\taxi\yellow_tripdata_2022-01.parquet, ... 11 other sources]
        PROJECT */19 COLUMNS
        ESTIMATED ROWS: 29567172
Time: 0.00s


In [19]:
lf = pl.LazyFrame(
    {"col1": [1, 2, 3], "col2": [4, 5, 6]}
)

print(lf.collect())

shape: (3, 2)
┌──────┬──────┐
│ col1 ┆ col2 │
│ ---  ┆ ---  │
│ i64  ┆ i64  │
╞══════╪══════╡
│ 1    ┆ 4    │
│ 2    ┆ 5    │
│ 3    ┆ 6    │
└──────┴──────┘


In [7]:
big_sales_data = pl.LazyFrame(
    {"sale_id": [101, 102, 103], "amount":[250, 150, 300]}
)

sales_metadata = pl.DataFrame(
    {
        "sale_id": [101, 102, 103], "category": ["A", "B", "A"]
    }
)

big_sales_data.join(sales_metadata.lazy(), on="sale_id").collect()
print(big_sales_data.join(sales_metadata.lazy(), on="sale_id").collect())

shape: (3, 3)
┌─────────┬────────┬──────────┐
│ sale_id ┆ amount ┆ category │
│ ---     ┆ ---    ┆ ---      │
│ i64     ┆ i64    ┆ str      │
╞═════════╪════════╪══════════╡
│ 101     ┆ 250    ┆ A        │
│ 102     ┆ 150    ┆ B        │
│ 103     ┆ 300    ┆ A        │
└─────────┴────────┴──────────┘
